In [1]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('enero25processed.csv', sep='|')

# Mostrar las primeras filas
print(df.head())

    id                   Autor/a                                 Obra  \
0  582  Bartolomé Torres Naharro                     Comedia Aquilana   
1  583  Bartolomé Torres Naharro                                  NaN   
2  584         Jerónimo Bermúdez  Nise, la tragedia de Inés de Castro   

          Fecha de estreno         Dirección                  Compañía  \
0       3 de marzo de 2018        Ana Zamora              Nao d´amores   
1      20 de marzo de 2018  Agustín Iglesias  Compañía Teatro Guirigái   
2  12 de diciembre de 2019        Ana Zamora              Nao d´amores   

                              Festivales  \
0  Festival de Teatro Clásico de Almagro   
1                                    NaN   
2  Festival de Teatro Clásico de Almagro   

                                    Lugar de estreno  
0  Teatro de la Comedia. Compañía Nacional de Tea...  
1  Teatro de la Comedia. Compañía Nacional de Tea...  
2                                Teatro de la Abadía  


In [2]:
# Crear una copia del dataframe
df_procesado = df.copy()

# Reemplazar valores vacíos con 'nan'
df_procesado = df_procesado.fillna('nan')

# Añadir la nueva columna con fecha por defecto
df_procesado['fechaFormat'] = '0-00-00'

# Guardar el nuevo CSV
df_procesado.to_csv('teatro_fechaFormat_enero25.csv', index=False, sep='|')

# Mostrar las primeras filas del nuevo dataframe para verificar
print(df_procesado.head())

    id                   Autor/a                                 Obra  \
0  582  Bartolomé Torres Naharro                     Comedia Aquilana   
1  583  Bartolomé Torres Naharro                                  nan   
2  584         Jerónimo Bermúdez  Nise, la tragedia de Inés de Castro   

          Fecha de estreno         Dirección                  Compañía  \
0       3 de marzo de 2018        Ana Zamora              Nao d´amores   
1      20 de marzo de 2018  Agustín Iglesias  Compañía Teatro Guirigái   
2  12 de diciembre de 2019        Ana Zamora              Nao d´amores   

                              Festivales  \
0  Festival de Teatro Clásico de Almagro   
1                                    nan   
2  Festival de Teatro Clásico de Almagro   

                                    Lugar de estreno fechaFormat  
0  Teatro de la Comedia. Compañía Nacional de Tea...     0-00-00  
1  Teatro de la Comedia. Compañía Nacional de Tea...     0-00-00  
2                               

In [3]:
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.llms import Ollama
import pandas as pd
import time

# Inicializar Ollama
llm = Ollama(model="llama2")

# Leer el CSV
df = pd.read_csv('teatro_fechaFormat_enero25.csv', sep='|')

# Función para procesar una fecha con Ollama
def procesar_fecha(fecha):
    try:
        prompt = f"formatea esta fecha en el formato dd-mm-yyyy, dame solo la fecha formateada, no quiero explicaciones: {fecha}"
        respuesta = llm.invoke(prompt)
        return respuesta.strip()
    except Exception as e:
        print(f"Error procesando fecha: {fecha}. Error: {e}")
        return '00-00-0000'

# Procesar cada fecha y guardar inmediatamente
for idx, row in df.iterrows():
    fecha_original = row['Fecha de estreno']
    fecha_procesada = procesar_fecha(fecha_original)
    df.at[idx, 'fechaFormat'] = fecha_procesada
    # Imprimir información de cada fila
    print(f"Fila {idx}: Original: {fecha_original} -> Procesada: {fecha_procesada}")
    # Guardar después de cada actualización
    df.to_csv('teatro_fechaOllama.csv', index=False, sep='|')

print("Proceso completado!")

/var/folders/3h/bml8f_c50dxgf6rhz_0bpn8r0000gn/T/ipykernel_9662/4067012247.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


Fila 0: Original: 3 de marzo de 2018 -> Procesada: 
Fila 1: Original: 20 de marzo de 2018 -> Procesada: 
Fila 2: Original: 12 de diciembre de 2019 -> Procesada: 
Proceso completado!


In [5]:
import pandas as pd
import re

# Leer el CSV
df = pd.read_csv('teatro_fechaOllama_enero25.csv', sep='|')

# Función para validar el formato de fecha
def validar_formato_fecha(fecha):
    # Patrón para dd-mm-yyyy
    patron = r'^\d{2}-\d{2}-\d{4}$'
    if pd.isna(fecha) or not isinstance(fecha, str):
        return pd.NA
    return fecha if re.match(patron, fecha.strip()) else pd.NA

# Aplicar la validación a la columna fechaFormat
df['fechaFormat'] = df['fechaFormat'].apply(validar_formato_fecha)

# Guardar el resultado
df.to_csv('teatro_fechaOllama_enero25.csv', index=False, sep='|')

# Mostrar un resumen de los cambios
print("Muestra de los datos actualizados:")
print(df[['fechaFormat']].head())
print("\nCantidad de fechas válidas:", df['fechaFormat'].notna().sum())
print("Cantidad de fechas inválidas (NaN):", df['fechaFormat'].isna().sum())

FileNotFoundError: [Errno 2] No such file or directory: 'teatro_fechaOllama_enero25.csv'

In [ ]:
import pandas as pd
import re

# Leer el CSV
df = pd.read_csv('teatro_fechaOllama_enero25.csv', sep='|')

# Función para validar el formato de fecha
def validar_formato_fecha(fecha):
    # Patrón para dd-mm-yyyy
    patron = r'^\d{2}-\d{2}-\d{4}$'
    if pd.isna(fecha) or not isinstance(fecha, str):
        return pd.NA
    return fecha if re.match(patron, fecha.strip()) else pd.NA

# Aplicar la validación a la columna fechaFormat
df['fechaFormat'] = df['fechaFormat'].apply(validar_formato_fecha)

# Crear un nuevo DataFrame solo con las columnas deseadas
df_fechas = df[['Fecha de estreno', 'fechaFormat']]

# Guardar el resultado
df_fechas.to_csv('teatro_fechas_comparacion.csv', index=False, sep='|')

# Mostrar un resumen de los cambios
print("Muestra de los datos guardados:")
print(df_fechas.head())
print("\nCantidad de fechas válidas:", df['fechaFormat'].notna().sum())
print("Cantidad de fechas inválidas (NaN):", df['fechaFormat'].isna().sum())